## Set up lightning-cli's

In [13]:
path_to_lightning = ! echo $PATH_TO_LIGHTNING
lightning_bin_dir = ! echo $LIGHTNING_BIN_DIR

# if "list index out of range", make sure you set where your nodes and lightning bins are
path_to_lightning = path_to_lightning[0]
lightning_bin_dir = lightning_bin_dir[0]

l1 = f'{lightning_bin_dir}/lightning-cli --lightning-dir={path_to_lightning}/l1'
l2 = f'{lightning_bin_dir}/lightning-cli --lightning-dir={path_to_lightning}/l2'

print("l1's lightning-cli is: ", l1)
print("l2's lightning-cli is: ", l2)


l1's lightning-cli is:  /nix/store/0bq00qd0scchy6vasjpqxqg53aijcca3-clightning-v23.11rc1/bin/lightning-cli --lightning-dir=/home/gudnuf/cashu/pyshu_mint/.lightning_nodes/l1
l2's lightning-cli is:  /nix/store/0bq00qd0scchy6vasjpqxqg53aijcca3-clightning-v23.11rc1/bin/lightning-cli --lightning-dir=/home/gudnuf/cashu/pyshu_mint/.lightning_nodes/l2


## Swapping 

"most important component of the Cashu system"

We will mint one 2 sat token, and then swap it for 2 1 sat tokens.

## Mint

see [mint_melt.ipynb](./mint_melt.ipynb)

In [14]:
import json
from coincurve import PrivateKey, PublicKey
from random import randint
from lib import construct_inputs, generate_blinded_messages

In [15]:
pubkeys = !$l1 cashu-get-keys | jq -r .keysets[0].keys
pubkeys = json.loads(''.join(pubkeys))
privkeys = !$l1 cashu-dev-get-privkeys
privkeys = json.loads(''.join(privkeys))

In [16]:
mint_quote = !$l1 -k cashu-quote-mint amount=2 unit="sat" | jq -r 
mint_quote = json.loads(''.join(mint_quote))

# quote is the unique id for this exchange
mint_id = mint_quote["quote"]
# `request` is a lightning invoice
mint_invoice = mint_quote["request"]

print("quoteId:", mint_id)
print("bolt11: ", mint_invoice)

quoteId: 2916c5f364b57bbc
bolt11:  lnbcrt20n1pj64u0dsp5w6jl4y2fdy07q48mp2wwa0sh0cyehghwras52l5ajxmp7mdrgjgqpp5dhjjgzrm9vapp5fhrcltj2vwhf6jjcz8l6q94j5wacsz93pl9hzqdqsg9hzq6twwehkjcm9xqyjw5qcqp2fp4p3lphkc0hkn2mya2dhhq7zegshxexaqqe3gt0sy3fgtnqmwgrkkfs9qx3qysgq3k0klxc7w8h2yzu5p08wq9tkwv6p8ytt7964k8w8xj2erpy38lcysr0j7arx6t7chx3tlyky8p0gnehwvzpapj3taem2y0844t6u5aqqsfjqpj


In [17]:
mint_invoice_status = !$l2 pay "$mint_invoice" | jq -r .status
print ("MINT INVOICE STATUS:", mint_invoice_status)

MINT INVOICE STATUS: ['complete']


In [18]:
secrets = [str(randint(0, 10000)) for _ in range(1)]
amounts = [2]
b_messages, rs = generate_blinded_messages(secrets, amounts)

json.dumps(b_messages)

'[{"amount": 2, "B_": "023dfa3e837abad7cf417cc760826dab5924d2428b58996c0f3b3801d54f0f91ad"}]'

### Mint token

In [20]:
# lightning-cli -k cashu-mint quote=<str> blinded_messages=<List[blinded_messages]>
mint_command = f'{l1} -k cashu-mint quote=\'{mint_id}\' blinded_messages=\'{json.dumps(b_messages)}\''
blinded_sigs = !$mint_command
blinded_sigs = json.loads(''.join(blinded_sigs))

# blinded_sigs should be a list
if isinstance(blinded_sigs, dict):
    raise AssertionError("{}".format(blinded_sigs))
else:
    print(blinded_sigs)

[{'amount': 2, 'id': '00f775c2e5e81aa3', 'C_': '02a2cc31e0b7c3c59d942ce2941c02609fdac54da9e5ff9d341ef466ca1f150333'}]


### Generate new blinded messages to swap


In [21]:
swap_secrets = [str(randint(0, 10000)) for _ in range(2)]
swap_amounts = [1, 1]
swap_b_messages, swap_rs = generate_blinded_messages(swap_secrets, swap_amounts)

inputs = construct_inputs(blinded_sigs,rs, secrets, pubkeys, privkeys)
inputs

'[{"amount": 2, "C": "030f5f32a90d057488242954d76f39f20dc2abea5724a9de8ee8f2fa7f899ade79", "id": "00f775c2e5e81aa3", "secret": "7800"}]'

In [22]:
swap_command = f'{l1} -k cashu-swap inputs=\'{inputs}\' outputs=\'{json.dumps(swap_b_messages)}\''
!$swap_command

[
   {
      "amount": 1,
      "id": "00f775c2e5e81aa3",
      "C_": "02be2f66739af75338e71e4761a9395128751433f93dbd7bb07fa0ac4b75355736"
   },
   {
      "amount": 1,
      "id": "00f775c2e5e81aa3",
      "C_": "022437dee8a0739654d83e1fa650dab0e52be6a200fa4243f95c2f3c6e67c72e4f"
   }
]
